In [1]:
import os
import shutil
import time
import pandas as pd
from tqdm.notebook import tqdm      #tqdm(filelist, total = len(filelist, position=0, leave=Ture)
import re
from pathlib import Path
import traceback
from os.path import join

### 나스에 파일 올리기

In [2]:
#[시작]###########################################################
#디렉토리 만들기
path = r'\\192.168.0.75\솔림헬프'
os.chdir(path)
docu_list = ["원인서류", "양도통지서", "집행권원", "강제집행", "등초본", "개인회생", "신용회복", "파산", "재산조사", "기타"]
sell_list = ["전북-01", "DNP-01", "DNP-02", "메리츠캐_01", "KNS", "DNP-03", "한울가람", "케이에스", \
    "유안타저-01", "KB카드-01", "베리타스-01", "BNK-01", "스마트저축-01", "우담-01", "고려저축-01", \
        "롯데카드-01", "헤럴드", "KCR", "아인스", "하나캐-02", "베델-01", "롯데카드-03", "하나캐-01", \
            "더키움", "롯데카드-02", "JT캐-02", "흥국화재-01", "롯데카드-04", "산은캐-01", "대산", "JT캐-01"]

for dir_ in docu_list :
    for sell_ in sell_list :
        os.makedirs(dir_+"/"+sell_, exist_ok=True)

#[끝]###########################################################

In [3]:
#[시작]파일업로드###########################################################
## 참조 df 불러오기 ~~~~ ":"와 공백 xlsx파일에서 미리 제거해둬야 ~~~~
## 참조할 col_name = '성명', '주민번호', '채무자키', '매각사', '보증인성명'
df_c = pd.read_excel(r'C:\Users\SL\Desktop\채무자조회.xlsx')
df_c.tail()

,채무자키,매각사구분,성명,주민번호인,성별,매각사,회생사건결과,관리자기타,보증인성명,변경전성명인,변경전주민번호인,보증인수,파산사건결과
67722,20459682,KB카드-01,YOON HYUN WOOK,671224-5600173,남자,KB국민카드,NaN,9013302712,NaN,NaN,NaN,0,NaN
67723,20459636,KB카드-01,YUTAIJI,830211-5140013,남자,KB국민카드,NaN,9013302093,NaN,NaN,NaN,0,NaN
67724,20447783,KB카드-01,ZAHER KAREEM ATIA,900803-5100144,남자,KB국민카드,NaN,9013302590,NaN,NaN,NaN,0,NaN
67725,20459338,KB카드-01,ZHANG FANJING,871111-6340392,여자,KB국민카드,NaN,9013202025,NaN,NaN,NaN,0,NaN
67726,20459630,KB카드-01,ZONG WEI,580805-6101696,여자,KB국민카드,NaN,9013302001,NaN,NaN,NaN,0,NaN


In [ ]:
dict_sell ={}
for ind, col in df_c.iterrows():
    dict_sell[str(col.채무자키)] = col.매각사구분

print(len(dict_sell))

In [ ]:
#파일 업로드
path = r'D:\부산 정리\대산대부'
f_d = r'\\192.168.0.75\솔림헬프'
os.chdir(path)
file_list = os.listdir(path)
index = 0
error = []
# 1) depth2 문서종류 매칭
docu_match = {"원인서류" : "원인서류", "채권양도통지서":"양도통지", "판결문":"법원서류", "지급명령":"법원서류", "이행권고":"법원서류", \
    "화해권고":"법원서류", "타채":"법원서류", "결정문":"법원서류", "등본":"초본등본", "초본":"초본등본", "개회":"신용회복", "신복":"신용회복", "파산":"신용회복"}

In [ ]:

for f in tqdm(file_list) : 
    depth1 = ""
    depth2 = ""
    depth3 = f[:8]
    
    # 문서종류(depth1)
    for key, value in docu_match.items() : 
        if re.search(key, f) :
            depth1 = value
            continue
        else :
            error.append(f)
    
    # 매각사(depth2) 
    depth2 = dict_sell[depth3]

    
    #디렉토리 및 파일명 작성
    new_name = f
    f_d_full = os.path.join(f_d, depth1,depth2,depth3)
    try :
        if not os.path.exists(f_d_full) :
            os.mkdir(f_d_full)
    except :
        print(traceback.format_exc())
        print(f_d_full)
    
    #이름 같은 파일은 넘버링
    temp = Path(f_d_full + "/" +new_name) #넘버링 없는 상태라 문제없음
    i = 1
    while os.path.exists(f_d_full+"/"+new_name): # os.path는 무조건 경로를 매개변수로..작업디렉토리가 아니므로 풀경로, new_name이 매개변수에 포함되어야 함!
        new_name = temp.stem + "_" +"("+str(i)+")"+temp.suffix
        i += 1
                    
    f_d_final = f_d_full + "/" + new_name
    shutil.copy2(f, f_d_final)
    index += 1

print(index, "개의 파일 업로드 완료")
os.chdir('C:/') # 작업디렉토리 옮겨줘야 폴더 삭제 가능

### 파일 비교 및 차집합

In [ ]:
myPath= r'D:\부산 정리_v2'
nasPath=r'\\192.168.0.75\스캔파일\새 스캔파일(업로드)'

In [ ]:
myFile = [f for f in os.scandir(myPath) if f.is_file()]

In [ ]:
nasFile = [f for f in os.scandir(nasPath) if f.is_file()]

In [ ]:
onlyLocal = list(set(myFile) - set(nasFile))
print(len(onlyLocal))

In [ ]:
onlynas = list(set(myFile) - set(nasFile))
print(len(onlynas))